<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
with open(f'./module1-rnn-and-lstm/100-0.txt', 'r') as f:
    text = f.read()

In [2]:
import re

text = re.sub(r'[^a-zA-Z0-9;\'\",-\[\]\(\)’\.\:\?\!\n]+',' ',text)
text = re.sub(r'(?=[;\'\",\[\]\(\)\.\:\?\!\n])',' ',text)
text = re.sub(r'(?<=[;\'\",\[\]\(\)\.\:\?\!\n])',' ',text)
text = re.sub(r'  +',' ',text)

In [3]:
text_words = text.split(' ')

In [4]:
wordslist = list(set(text_words))

In [5]:
import pandas
counts = pandas.Series(text.split(' ')).value_counts()
counts

\n              166902
,                92563
.                84177
the              25553
I                22445
                 ...  
Pickt-hatch          1
Gored                1
recognizance         1
ice-brook’s          1
love-broker          1
Length: 38910, dtype: int64

In [6]:
wordslist = list(counts[:511].index)

In [7]:




word_indices = {w:i for i,w in enumerate(wordslist)}
indices_word = {i:w for i,w in enumerate(wordslist)}
if '' not in word_indices:
        indices_word[max(word_indices.values()) + 1] = ''
        word_indices[''] = max(word_indices.values()) + 1
word_indices

{'\n': 0,
 ',': 1,
 '.': 2,
 'the': 3,
 'I': 4,
 'and': 5,
 ';': 6,
 'to': 7,
 'of': 8,
 "'": 9,
 'a': 10,
 'you': 11,
 'my': 12,
 '?': 13,
 'in': 14,
 'is': 15,
 'that': 16,
 'not': 17,
 '!': 18,
 'And': 19,
 'me': 20,
 'with': 21,
 'it': 22,
 'his': 23,
 'be': 24,
 'your': 25,
 'for': 26,
 'this': 27,
 'have': 28,
 'him': 29,
 'he': 30,
 'd': 31,
 'thou': 32,
 'will': 33,
 ':': 34,
 'as': 35,
 'The': 36,
 'so': 37,
 'her': 38,
 'thy': 39,
 'but': 40,
 's': 41,
 'all': 42,
 'To': 43,
 'do': 44,
 'are': 45,
 'thee': 46,
 'shall': 47,
 '[': 48,
 ']': 49,
 'by': 50,
 'on': 51,
 'That': 52,
 'no': 53,
 'our': 54,
 'But': 55,
 'we': 56,
 'from': 57,
 'What': 58,
 'at': 59,
 'O': 60,
 'good': 61,
 'what': 62,
 'A': 63,
 'more': 64,
 'am': 65,
 'their': 66,
 'was': 67,
 'she': 68,
 'would': 69,
 'they': 70,
 'love': 71,
 'now': 72,
 'Enter': 73,
 'them': 74,
 'lord': 75,
 'if': 76,
 'For': 77,
 'or': 78,
 'man': 79,
 'sir': 80,
 'us': 81,
 'If': 82,
 'one': 83,
 'hath': 84,
 'My': 85,
 'here

In [8]:
max_len = 128

In [9]:
# text_sequences = text.split('\n')
text_sequences = []
step = 67
for i in range(0, len(text_words) - max_len, step):
    seq = [w for w in text_words[i: i + max_len] if w in wordslist]
    while len(seq) < 11: seq.append('')
    
    text_sequences.append(seq)
# text_sequences = [s for s in word_sequences if len(s) > 0 and len(s) <= max_len]
text_sequences[1206]

['Is',
 'not',
 'this',
 'a',
 'strange',
 'fellow',
 ',',
 'my',
 'lord',
 ',',
 '\n',
 'that',
 'so',
 'seems',
 'to',
 'this',
 'business',
 ',',
 'which',
 'he',
 'knows',
 'is',
 '\n',
 'not',
 'to',
 'be',
 'done',
 ';',
 'himself',
 'to',
 'do',
 ',',
 'and',
 'better',
 'be',
 'than',
 'to',
 '\n',
 '.',
 '\n',
 '\n',
 'SECOND',
 'LORD',
 '.',
 '\n',
 'You',
 'do',
 'not',
 'know',
 'him',
 ',',
 'my',
 'lord',
 ',',
 'as',
 'we',
 'do',
 ';',
 'certain',
 'it',
 'is',
 'that',
 'he',
 'will',
 '\n',
 'himself',
 'into',
 'a',
 'favour',
 ',',
 'and',
 'for',
 'a',
 'a',
 'great',
 '\n']

In [10]:
len(text_sequences)

37401

In [11]:
word_sequences = [
    [word_indices[w] for w in text_sequence]
        for text_sequence in text_sequences
]
for word_sequence in word_sequences:
    while len(word_sequence) < max_len:
        word_sequence.append(word_indices[''])
word_sequences[1206]

[152,
 17,
 27,
 10,
 483,
 417,
 1,
 12,
 75,
 1,
 0,
 16,
 37,
 924,
 7,
 27,
 526,
 1,
 99,
 30,
 569,
 15,
 0,
 17,
 7,
 24,
 190,
 6,
 280,
 7,
 44,
 1,
 5,
 218,
 24,
 101,
 7,
 0,
 2,
 0,
 0,
 266,
 316,
 2,
 0,
 90,
 44,
 17,
 88,
 29,
 1,
 12,
 75,
 1,
 35,
 56,
 44,
 6,
 673,
 22,
 15,
 16,
 30,
 33,
 0,
 280,
 225,
 10,
 832,
 1,
 5,
 26,
 10,
 10,
 161,
 0,
 1023,
 1023,
 1023,
 1023,
 1023,
 1023,
 1023,
 1023,
 1023,
 1023,
 1023]

In [12]:
indices_word[64]

'more'

In [13]:
import numpy as np

x = np.zeros((len(word_sequences), max_len, len(word_indices)), dtype=np.bool)
y = np.zeros((len(word_sequences), len(word_indices)), dtype=np.bool)

for i, sequence in enumerate(word_sequences):
    if i == len(word_sequences) - 1: break

    try:
        for t, word in enumerate(sequence):
            x[i,t,word] = 1
    except Exception:
        print(i, sequence)
        raise
        
    y[i, word_sequences[i+1][0]] = 1

In [14]:
len(word_indices)

1024

In [15]:
word_indices['']

1023

In [16]:
indices_word[0]

'\n'

In [17]:
text_sequences[0]

['\n',
 'The',
 'of',
 ',',
 'by',
 '\n',
 '\n',
 '\n',
 'This',
 'is',
 'for',
 'the',
 'use',
 'of',
 'in',
 'the',
 'and',
 '\n',
 'most',
 'other',
 'parts',
 'of',
 'the',
 'world',
 'at',
 'no',
 'and',
 'with',
 'almost',
 'no',
 '\n',
 '.',
 'You',
 'may',
 'it',
 ',',
 'give',
 'it',
 'away',
 'or',
 'it',
 'under',
 'the',
 '\n',
 'of',
 'the',
 'with',
 'this',
 'or',
 'at',
 '\n',
 '.',
 '.',
 '.',
 'If',
 'you',
 'are',
 'not']

In [18]:
max([v for v in indices_word.keys()])

1023

In [19]:
max([v for v in word_indices.values()])

1023

In [20]:
x[1][0]

array([False, False, False, ..., False, False, False])

In [21]:
y[0]

array([False, False, False, ..., False, False, False])

In [22]:
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(LSTM(128,input_shape=(max_len, len(word_indices))))
model.add(Dense(len(word_indices), activation='softmax'))

          
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [23]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [24]:
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text_words) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text_words[start_index: start_index + max_len]
        generated += ' '.join(sentence)
        print('----- Generating with seed: "' + ' '.join(sentence) + '"')
        # sys.stdout.write(generated)

        for i in range(800):
            x_pred = np.zeros((1, max_len, len(word_indices)))
            for t, word in enumerate(sentence[-max_len:]):
                if word in word_indices:
                    ind = word_indices[word]
                else:
                    ind = word_indices['']
                x_pred[0, t, ind] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence.append(next_word)

            if i%80 == 0:
                sys.stdout.write(str(i//80))
                sys.stdout.flush()
        # print(str(sentence))
        print(' '.join(sentence) + '\n')

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [25]:
model.fit(x, y,
          batch_size=512,
          epochs=8,
          callbacks=[print_callback])

very ten that they make the devils mar five . 
 CLEOPATRA . . with saw next ' 
 , spirit was . in Prince your 
 in ' . . for ; , will then must , , , best 
 
 : : Come thee tears others , chamber you dead that are 
 , so ? ! 
 with Re-enter am 
 . And him at 
 times is ? 
 , ? like than 
 me and , 
 you and her ; of from this 
 
 on comfort same I as this with , 
 
 of 
 , of . the , for do . this . loves and ' . them of A now d you the so the , , and o’ , tears and thee , and act Marry not part night out , else 
 and of 
 
 I mark of 
 
 I , I , indeed Who do , general that 
 in as , come Duke 
 loss can sing . in Aside 
 tis would thee her he to for s like else answer love me 
 this , thou . , when . , you ' love in . , Is 
 , 
 to friend , I , His 
 a 
 in even him him 
 them , turn is . . th a world my 
 , pity Duke get God it born never my of take of , . to 
 well [ but again the PEDRO here DUKE 
 me is in the and yourself ? , of news . you made should thyself on 
 
 love 
 : worl

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN